In [1]:
from unicodedata import name
from keras.models import Sequential
from keras.layers import Dense


In [2]:
newmodel = Sequential()

# Defining the Input layer and FIRST hidden layer, both are same!
newmodel.add(Dense(units=5, input_dim=7, kernel_initializer='normal', activation='relu'))

# Defining the Second layer of the model
# after the first layer we don't have to specify input_dim as keras configure it automatically
newmodel.add(Dense(units=6, kernel_initializer='normal', activation='tanh'))

# The output neuron is a single fully connected node 
# Since we will be predicting a single number
newmodel.add(Dense(1, kernel_initializer='normal'))


In [3]:
newmodel.load_weights('weights')

In [4]:
newmodel.get_weights()

[array([[ 0.26018   , -0.25986463, -0.26137364,  0.27419063,  0.21564314],
        [ 0.22583991, -0.24365923, -0.2533461 ,  0.24800245,  0.2714464 ],
        [-0.18252711,  0.24468404,  0.26175734, -0.07091657, -0.14453416],
        [-0.16009206,  0.10525325,  0.11674461, -0.03122249, -0.10320532],
        [ 0.06267378,  0.12009878,  0.06300337, -0.056673  , -0.01808654],
        [ 0.03777277,  0.04253545,  0.03647057, -0.01638531, -0.05307421],
        [ 0.00817827,  0.10213455,  0.02497949, -0.07986367, -0.11653981]],
       dtype=float32),
 array([0.25600025, 0.20919976, 0.20819639, 0.25541753, 0.2459631 ],
       dtype=float32),
 array([[-0.19991107, -0.28414714,  0.26100773, -0.30522478, -0.2732377 ,
         -0.11481238],
        [ 0.21360853,  0.3146705 , -0.23762776,  0.19925158,  0.2648562 ,
         -0.09506173],
        [ 0.18625975,  0.26122814, -0.28725618,  0.15753506,  0.25604692,
         -0.3618276 ],
        [-0.25932366, -0.20212226,  0.25030184, -0.20273493, -0.3425

In [5]:
newmodel.compile(loss='mean_squared_error', optimizer='adam')

In [6]:
# Reading the cleaned numeric car prices data
import pandas as pd
import numpy as np

# To remove the scientific notation from numpy arrays
np.set_printoptions(suppress=True)

CarPricesDataNumeric=pd.read_pickle('CarPricesData.pkl')
CarPricesDataNumeric.head()

,Age,KM,Weight,HP,MetColor,CC,Doors,Price
0,23.0,46986,1165.0,90,1,2000.0,3,13500
1,23.0,72937,1165.0,90,1,2000.0,3,13750
2,24.0,41711,1165.0,90,1,2000.0,3,13950
3,26.0,48000,1165.0,90,0,2000.0,3,14950
4,30.0,38500,1170.0,90,0,2000.0,3,13750


In [7]:
# Separate Target Variable and Predictor Variables
TargetVariable=['Price']
Predictors=['Age', 'KM', 'Weight', 'HP', 'MetColor', 'CC', 'Doors']

X=CarPricesDataNumeric[Predictors].values
y=CarPricesDataNumeric[TargetVariable].values

### Sandardization of data ###
from sklearn.preprocessing import StandardScaler
PredictorScaler=StandardScaler()
TargetVarScaler=StandardScaler()

# Storing the fit object for later reference
PredictorScalerFit=PredictorScaler.fit(X)
TargetVarScalerFit=TargetVarScaler.fit(y)

# Generating the standardized values of X and y
X=PredictorScalerFit.transform(X)
y=TargetVarScalerFit.transform(y)

# Split the data into training and testing set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Quick sanity check with the shapes of Training and testing datasets
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(1004, 7)
(1004, 1)
(431, 7)
(431, 1)


In [8]:
for _ in range(5):
    newmodel.fit(X_train, y_train, batch_size = 15, epochs = 1, verbose=1)

67/67 [==============================] - 0s 1ms/step - loss: 0.2876


In [9]:
newmodel.get_weights()

[array([[ 0.4499867 , -0.47380415, -0.478202  ,  0.46762845,  0.41546598],
        [ 0.19013187, -0.32406485, -0.3297185 ,  0.19251637,  0.2962058 ],
        [-0.20942305,  0.41499114,  0.4310645 , -0.08158383, -0.18330014],
        [-0.14473608,  0.10811359,  0.13318077,  0.02520574, -0.12093213],
        [ 0.1452367 ,  0.12441258,  0.0614786 , -0.00895103,  0.05304672],
        [ 0.12945853, -0.01544089, -0.01152006,  0.0935207 ,  0.02598571],
        [ 0.06745277, -0.00010603, -0.07501119,  0.00722132, -0.09750124]],
       dtype=float32),
 array([0.4158796 , 0.25507063, 0.2633755 , 0.4188899 , 0.38858286],
       dtype=float32),
 array([[-0.32872748, -0.41563186,  0.39225218, -0.43247518, -0.40204775,
          0.05795834],
        [ 0.3790105 ,  0.44131505, -0.37650836,  0.3718593 ,  0.40241948,
         -0.26159   ],
        [ 0.35451862,  0.3923    , -0.43008924,  0.33274844,  0.3973808 ,
         -0.53231525],
        [-0.38782635, -0.33277836,  0.3808933 , -0.32894152, -0.4716

In [10]:
import tf2onnx
output = "updatedANN.onnx"
saved = tf2onnx.convert.from_keras(newmodel, output_path = output)

Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
